<a href="https://colab.research.google.com/github/jharilal/candlestick_analysis/blob/main/Candlestick_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Candlestick Indicator Analysis



## Outline



1. Problem Questions
2. Project Installs
3. Data Collection
4. Data Preparation
5. Data Analysis
6. Report

## Problem Questions

 - How accurate are technical indicators in predicting stock price movement for the top 100 / 500 companies over the past 5 years?
 - Does the industry / sub industry demonstrate a different amount of success with indicators?
 - Analyze the popular, single candle-stick indicators - Hammer /  Long Legged Doji / Shooting Star / Hangman / Gravestone / Dragonfly
 - Build this in a way such that it's possible to expand on the technical indicator of analysis
 - How often do these indicators occur?

 These questions help answer the measurable reliability of using these indicators to influence trading decisions.


## Project Installs

In [1]:
!pip install yahoofinance
!pip install yfinance

     |████████████████████████████████| 6.4 MB 6.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import yfinance as yf
import pandas as pd
msft_tick = yf.download('MSFT')

[*********************100%***********************]  1 of 1 completed


## Data Collection

In [3]:
nasdaq_df = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')

In [4]:
nasdaq_df[3]

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care,Health Care Supplies
...,...,...,...,...
97,Workday,WDAY,Information Technology,Application Software
98,Xcel Energy,XEL,Utilities,Multi-Utilities
99,Xilinx,XLNX,Information Technology,Semiconductors
100,Zoom,ZM,Information Technology,Application Software


In [5]:
n100 = nasdaq_df[3]

In [6]:
n100

,Company,Ticker,GICS Sector,GICS Sub-Industry
0,Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
1,Adobe,ADBE,Information Technology,Application Software
2,ADP,ADP,Information Technology,Data Processing & Outsourced Services
3,Airbnb,ABNB,Consumer Discretionary,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care,Health Care Supplies
...,...,...,...,...
97,Workday,WDAY,Information Technology,Application Software
98,Xcel Energy,XEL,Utilities,Multi-Utilities
99,Xilinx,XLNX,Information Technology,Semiconductors
100,Zoom,ZM,Information Technology,Application Software


In [7]:
ticker_sec_gics = pd.DataFrame()
ticker_sec_gics = n100[['Ticker','Company','GICS Sector', 'GICS Sub-Industry']]

In [8]:
ticker_sec_gics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Ticker             102 non-null    object
 1   Company            102 non-null    object
 2   GICS Sector        102 non-null    object
 3   GICS Sub-Industry  102 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [9]:
ticker = ticker_sec_gics['Ticker'].to_list()

In [10]:
start_date = '2020-03-31'
end_date = '2021-04-01'

In [11]:
main_df = pd.DataFrame()

In [16]:
ticker = ['FB', 'AAPL', 'NFLX', 'GOOGL', 'AMZN']

In [17]:
for sym in ticker:

  temp_df = yf.download('{}'.format(sym), start=start_date, end=end_date)
  temp_df = temp_df.drop(columns=['Adj Close'], axis=1)

  temp_industry = ticker_sec_gics[ticker_sec_gics['Ticker'] == '{}'.format(sym)]['GICS Sector'].to_list()[0]
  temp_subindustry = ticker_sec_gics[ticker_sec_gics['Ticker'] == '{}'.format(sym)]['GICS Sub-Industry'].to_list()[0]

  temp_df.insert(0, 'Ticker', '{}'.format(sym))
  temp_df.insert(1, 'GICS Sector', '{}'.format(temp_industry))
  temp_df.insert(2, 'GICS Sub-Industry', '{}'.format(temp_subindustry))

  main_df = pd.concat([main_df, temp_df])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [18]:
main_df

,Ticker,GICS Sector,GICS Sub-Industry,Open,High,Low,Close,Volume
Date,,,,,,,,
2020-03-31,FB,Communication Services,Interactive Media & Services,165.479996,170.929993,164.199997,166.800003,23676300
2020-04-01,FB,Communication Services,Interactive Media & Services,161.619995,164.149994,158.039993,159.600006,19507600
2020-04-02,FB,Communication Services,Interactive Media & Services,159.100006,161.350006,155.919998,158.190002,20886300
2020-04-03,FB,Communication Services,Interactive Media & Services,157.149994,157.910004,150.830002,154.179993,25983300
2020-04-06,FB,Communication Services,Interactive Media & Services,160.149994,166.199997,158.509995,165.550003,28453600
...,...,...,...,...,...,...,...,...
2021-03-25,AMZN,Consumer Discretionary,Internet & Direct Marketing Retail,3072.989990,3109.780029,3037.139893,3046.260010,3563500
2021-03-26,AMZN,Consumer Discretionary,Internet & Direct Marketing Retail,3044.060059,3056.659912,2996.000000,3052.030029,3312900
2021-03-29,AMZN,Consumer Discretionary,Internet & Direct Marketing Retail,3055.439941,3091.250000,3028.449951,3075.729980,2746000


## Data Preparation

Code needs to::

iterate rows:

  -do basic calculation to determine presence of indicator

  -if indicator found, move on
  
  -save indicator in another dataframe with:

    -ticker, gics sec, gics sub-industry, indicator type, close, volume, uptrend/downtrend/neither, close value in 1d, 3d, 7d, 10d.

In [73]:
class Indicator:

  indicator_data = []
  
  @staticmethod
  def indicator_detect():
    pass

In [41]:
i = 0
for index, row in main_df.iterrows():
  o, h, l, c = row[3:-1]
  i +=1 
  if i == 1:
    break

In [ ]:
class Downtrend:
  pass

In [ ]:
class Uptrend:
  pass

hamer criteria: 


*   high ~ open / close value
*   low > 2x the lower bound



In [71]:
a = [10, 20]
b = 20

wick_check = Hammer.upper_wick_check(a, b)
wick_check

True

In [68]:
class Hammer:
  """Identifies and creates a Hammer Object"""

  def __init__(self, open, high, low, close, volume):
    self.open = open
    self.close = close
    self.high = high
    self.low = low
    self.volume = volume

  def hammer_

  def hammer_detect(open, high, low, close):

    upper_bound, lower_bound = max(open, close), min(open, close)
    upper_range = [upper_bound * 0.999, upper_bound * 1.001]
    day_diff = upper_bound - lower_bound


    check_one = Hammer.upper_wick_check()
    check_two = Hammer.lower_wick_check()
    check_three = 

    if check_one == True and check_two == True:
      return True
  
  def upper_wick_check(upper_range: list, high: float):
    if high >= upper_range[0] and high <= upper_range[1]:
      return True

  def lower_wick_check(day_diff, lower_bound, low):
    if (lower_bound - low) >= (2 * day_diff):
      return True

    

In [ ]:
class ShootingStar:
  """Identifies and creates a ShootingStar Object"""

In [ ]:
# class Longlegdoji:
#   pass

In [ ]:
# class Gravestone:
#   # pass

In [ ]:
# class Dragonfly:
#   pass